In [259]:
import pandas as pd
import re

In [435]:
df_recipe = pd.read_csv('RAW_recipes.csv')
df_review = pd.read_csv('RAW_interactions.csv')

In [436]:
df_recipe.describe()

,id,minutes,contributor_id,n_steps,n_ingredients
count,231637.000000,2.316370e+05,2.316370e+05,231637.000000,231637.000000
mean,222014.708984,9.398546e+03,5.534885e+06,9.765499,9.051153
std,141206.635626,4.461963e+06,9.979141e+07,5.995128,3.734796
min,38.000000,0.000000e+00,2.700000e+01,0.000000,1.000000
25%,99944.000000,2.000000e+01,5.690500e+04,6.000000,6.000000
50%,207249.000000,4.000000e+01,1.736140e+05,9.000000,9.000000
75%,333816.000000,6.500000e+01,3.982750e+05,12.000000,11.000000
max,537716.000000,2.147484e+09,2.002290e+09,145.000000,43.000000


In [456]:
df_recipe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231637 entries, 0 to 231636
Data columns (total 12 columns):
name              231636 non-null object
id                231637 non-null int64
minutes           231637 non-null int64
contributor_id    231637 non-null int64
submitted         231637 non-null object
tags              231637 non-null object
nutrition         231637 non-null object
n_steps           231637 non-null int64
steps             231637 non-null object
description       226658 non-null object
ingredients       231637 non-null object
n_ingredients     231637 non-null int64
dtypes: int64(5), object(7)
memory usage: 21.2+ MB


In [457]:
#check on NaN name

df_recipe[df_recipe['name'].isnull()]

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
721,NaN,368257,10,779451,2009-04-27,"['15-minutes-or-less', 'time-to-make', 'course...","[1596.2, 249.0, 155.0, 0.0, 2.0, 112.0, 14.0]",6,"['in a bowl , combine ingredients except for o...",-------------,"['lemon', 'honey', 'horseradish mustard', 'gar...",10


In [458]:
#check on NaN description

df_recipe[df_recipe['description'].isnull()].head(5)

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
5,apple a day milk shake,5289,0,1533,1999-12-06,"['15-minutes-or-less', 'time-to-make', 'course...","[160.2, 10.0, 55.0, 3.0, 9.0, 20.0, 7.0]",4,"['combine ingredients in blender', 'cover and ...",NaN,"['milk', 'vanilla ice cream', 'frozen apple ju...",4
8,bananas 4 ice cream pie,70971,180,102353,2003-09-10,"['weeknight', 'time-to-make', 'course', 'main-...","[4270.8, 254.0, 1306.0, 111.0, 127.0, 431.0, 2...",8,"['crumble cookies into a 9-inch pie plate , or...",NaN,"['chocolate sandwich style cookies', 'chocolat...",6
74,philly waldorf salad,5060,60,1534,1999-12-01,"['60-minutes-or-less', 'time-to-make', 'course...","[180.7, 22.0, 29.0, 3.0, 6.0, 33.0, 3.0]",4,"['combine softened cream cheese , orange juice...",NaN,"['philadelphia cream cheese', 'orange juice', ...",7
76,pizza stuffed potato,52443,25,1533,2003-01-28,"['30-minutes-or-less', 'time-to-make', 'course...","[183.3, 12.0, 9.0, 15.0, 21.0, 23.0, 6.0]",8,"['preheat oven to 450 degrees', 'cut potato in...",NaN,"['baking potato', 'mozzarella cheese', 'tomato...",7
99,the woiks dilly burgers,34930,32,23302,2002-07-24,"['bacon', '60-minutes-or-less', 'time-to-make'...","[449.2, 50.0, 16.0, 19.0, 58.0, 67.0, 2.0]",8,"['in a bowl , mix together the ground beef , m...",NaN,"[""mccormick's montreal brand steak seasoning"",...",9


In [459]:
#Remove NaN names and fill NaN descriptions from DataFrame and 

df_recipe = df_recipe[df_recipe['name'].notnull()].fillna(0)

df_recipe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 231636 entries, 0 to 231636
Data columns (total 12 columns):
name              231636 non-null object
id                231636 non-null int64
minutes           231636 non-null int64
contributor_id    231636 non-null int64
submitted         231636 non-null object
tags              231636 non-null object
nutrition         231636 non-null object
n_steps           231636 non-null int64
steps             231636 non-null object
description       231636 non-null object
ingredients       231636 non-null object
n_ingredients     231636 non-null int64
dtypes: int64(5), object(7)
memory usage: 23.0+ MB


In [460]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/brandon.arcilla/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/brandon.arcilla/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [461]:
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 

In [462]:
#lowercase text
def lowercase(text):
    text = text.lower()
    return text

In [463]:
#test string to see how each function is working
test = df_recipe['steps'][0]

test = lowercase(test)

print(test)

['make a choice and proceed with recipe', 'depending on size of squash , cut into half or fourths', 'remove seeds', 'for spicy squash , drizzle olive oil or melted butter over each cut squash piece', 'season with mexican seasoning mix ii', 'for sweet squash , drizzle melted honey , butter , grated piloncillo over each cut squash piece', 'season with sweet mexican spice mix', 'bake at 350 degrees , again depending on size , for 40 minutes up to an hour , until a fork can easily pierce the skin', 'be careful not to burn the squash especially if you opt to use sugar or butter', 'if you feel more comfortable , cover the squash with aluminum foil the first half hour , give or take , of baking', 'if desired , season with salt']


In [464]:
#remove special characters
def remove_special_characters(text):
    text = re.sub(r'[^a-z0-9]', ' ', text) #decided not to remove numbers as it could be important for temperatures
    return text

In [465]:
test = remove_special_characters(test)

print(test)

  make a choice and proceed with recipe    depending on size of squash   cut into half or fourths    remove seeds    for spicy squash   drizzle olive oil or melted butter over each cut squash piece    season with mexican seasoning mix ii    for sweet squash   drizzle melted honey   butter   grated piloncillo over each cut squash piece    season with sweet mexican spice mix    bake at 350 degrees   again depending on size   for 40 minutes up to an hour   until a fork can easily pierce the skin    be careful not to burn the squash especially if you opt to use sugar or butter    if you feel more comfortable   cover the squash with aluminum foil the first half hour   give or take   of baking    if desired   season with salt  


In [466]:
#remove stopwords
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    token = word_tokenize(text)
    remove_output = ' '.join(x for x in token if x not in stop_words)
    return remove_output

In [467]:
test = remove_stopwords(test)

print(test)

make choice proceed recipe depending size squash cut half fourths remove seeds spicy squash drizzle olive oil melted butter cut squash piece season mexican seasoning mix ii sweet squash drizzle melted honey butter grated piloncillo cut squash piece season sweet mexican spice mix bake 350 degrees depending size 40 minutes hour fork easily pierce skin careful burn squash especially opt use sugar butter feel comfortable cover squash aluminum foil first half hour give take baking desired season salt


In [468]:
#lemmatize words
def lemmatize(text):
    lemmatizer = WordNetLemmatizer()
    lemmatized_output = ' '.join(lemmatizer.lemmatize(x, 'v') for x in word_tokenize(text))
    return lemmatized_output

In [469]:
test = lemmatize(test)

print(test)

make choice proceed recipe depend size squash cut half fourths remove seed spicy squash drizzle olive oil melt butter cut squash piece season mexican season mix ii sweet squash drizzle melt honey butter grate piloncillo cut squash piece season sweet mexican spice mix bake 350 degrees depend size 40 minutes hour fork easily pierce skin careful burn squash especially opt use sugar butter feel comfortable cover squash aluminum foil first half hour give take bake desire season salt


In [470]:
#function to clean text
def text_cleaning(text):
    processed_text = []
    
    text = lowercase(text)
    
    text = remove_special_characters(text)
    
    text = remove_stopwords(text)
    
    text = lemmatize(text)
    
    processed_text.append(text)

    return processed_text

In [471]:
test2 = df_recipe['steps'][1]

print(test2)

print(text_cleaning(test2))

['preheat oven to 425 degrees f', 'press dough into the bottom and sides of a 12 inch pizza pan', 'bake for 5 minutes until set but not browned', 'cut sausage into small pieces', 'whisk eggs and milk in a bowl until frothy', 'spoon sausage over baked crust and sprinkle with cheese', 'pour egg mixture slowly over sausage and cheese', 's& p to taste', 'bake 15-20 minutes or until eggs are set and crust is brown']
['preheat oven 425 degrees f press dough bottom side 12 inch pizza pan bake 5 minutes set brown cut sausage small piece whisk egg milk bowl frothy spoon sausage bake crust sprinkle cheese pour egg mixture slowly sausage cheese p taste bake 15 20 minutes egg set crust brown']


In [472]:
df_recipe['clean_name'] = df_recipe['name'].apply(lambda x: text_cleaning(x))
df_recipe['clean_steps'] = df_recipe['steps'].apply(lambda x: text_cleaning(x))
df_recipe['clean_ingredients'] = df_recipe['ingredients'].apply(lambda x: text_cleaning(x))

In [473]:
df_recipe.columns

Index(['name', 'id', 'minutes', 'contributor_id', 'submitted', 'tags',
       'nutrition', 'n_steps', 'steps', 'description', 'ingredients',
       'n_ingredients', 'clean_name', 'clean_steps', 'clean_ingredients'],
      dtype='object')

In [474]:
df_clean = df_recipe[['id', 
                       'submitted',
                       'minutes', 
                       'name',
                       'clean_name', 
                       'clean_steps', 
                       'n_steps', 
                       'clean_ingredients', 
                       'n_ingredients']]

In [475]:
df_clean.head()

,id,submitted,minutes,name,clean_name,clean_steps,n_steps,clean_ingredients,n_ingredients
0,137739,2005-09-16,55,arriba baked winter squash mexican style,[arriba bake winter squash mexican style],[make choice proceed recipe depend size squash...,11,[winter squash mexican season mix spice honey ...,7
1,31490,2002-06-17,30,a bit different breakfast pizza,[bite different breakfast pizza],[preheat oven 425 degrees f press dough bottom...,9,[prepare pizza crust sausage patty egg milk sa...,6
2,112140,2005-02-25,130,all in the kitchen chili,[kitchen chili],[brown grind beef large pot add chop onions gr...,6,[grind beef yellow onions dice tomatoes tomato...,13
3,59389,2003-04-14,45,alouette potatoes,[alouette potatoes],[place potatoes large pot lightly salt water b...,11,[spreadable cheese garlic herbs new potatoes s...,11
4,44061,2002-10-25,190,amish tomato ketchup for canning,[amish tomato ketchup can],[mix ingredients boil 2 1 2 hours thick pour j...,5,[tomato juice apple cider vinegar sugar salt p...,8


In [23]:
df_review.describe()

,user_id,recipe_id,rating
count,1.132367e+06,1.132367e+06,1.132367e+06
mean,1.384291e+08,1.608972e+05,4.411016e+00
std,5.014269e+08,1.303987e+05,1.264752e+00
min,1.533000e+03,3.800000e+01,0.000000e+00
25%,1.354700e+05,5.425700e+04,4.000000e+00
50%,3.309370e+05,1.205470e+05,5.000000e+00
75%,8.045500e+05,2.438520e+05,5.000000e+00
max,2.002373e+09,5.377160e+05,5.000000e+00


In [24]:
df_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1132367 entries, 0 to 1132366
Data columns (total 5 columns):
user_id      1132367 non-null int64
recipe_id    1132367 non-null int64
date         1132367 non-null object
rating       1132367 non-null int64
review       1132198 non-null object
dtypes: int64(3), object(2)
memory usage: 43.2+ MB


In [432]:
#Check NaN reviews
df_review[df_review.review.isna()].head()

,user_id,recipe_id,date,rating,review
4141,2000971998,169916,2018-02-21,5,NaN
8206,2001494017,26110,2017-09-29,5,NaN
11235,2001434346,228641,2017-08-05,5,NaN
15813,2002307795,93886,2018-10-21,5,NaN
20448,2002265388,139918,2018-09-03,4,NaN


In [437]:
df_review = df_review[df_review['review'].notnull()]

In [439]:
df_review.rating.value_counts()

5    816229
4    187333
0     60847
3     40852
2     14122
1     12815
Name: rating, dtype: int64

In [440]:
#check reviews that have a 0 rating
df_review[df_review['rating']==0].head()

,user_id,recipe_id,date,rating,review
6,124416,120345,2011-08-06,0,"Just an observation, so I will not rate. I fo..."
12,468945,134728,2008-02-20,0,Made my own buttermilk w/ vinegar and milk. U...
23,681408,225241,2008-03-14,0,This is a very good recipe. We also want to c...
39,2002068522,29093,2018-03-26,0,i made it and it was amazing
56,980899,79222,2010-10-18,0,"I used half cream and half 1% milk, added anot..."


In [441]:
df_review['clean_review'] = df_review['review'].apply(lambda x: text_cleaning(x))

df_review.head()

,user_id,recipe_id,date,rating,review,clean_review
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...,[great salad cook top stave 15 minutes add sha...
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall...",[simple delicious great chilly fall even doubl...
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...,[work well easy use quite whole package 10oz w...
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...,[make mexican top take bunko everyone love]
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin...",[make cheddar bacon top add sprinkle black pep...


In [476]:
#Checking to see how many unique recipe ids in both DataFrames

print('df_review:', len(df_review.recipe_id.unique()))
print('df_recipe:', len(df_recipe.id.unique()))

df_review: 231630
df_recipe: 231636


In [99]:
avg_rating = df_review[['recipe_id', 'rating']].groupby('recipe_id').mean()

In [96]:
df_merge = df_recipe.merge(avg_rating, left_on='id', right_on='recipe_id')

In [77]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 231637 entries, 0 to 231636
Data columns (total 13 columns):
name              231636 non-null object
id                231637 non-null int64
minutes           231637 non-null int64
contributor_id    231637 non-null int64
submitted         231637 non-null object
tags              231637 non-null object
nutrition         231637 non-null object
n_steps           231637 non-null int64
steps             231637 non-null object
description       226658 non-null object
ingredients       231637 non-null object
n_ingredients     231637 non-null int64
rating            231637 non-null float64
dtypes: float64(1), int64(5), object(7)
memory usage: 24.7+ MB


In [78]:
df_merge.describe()

,id,minutes,contributor_id,n_steps,n_ingredients,rating
count,231637.000000,2.316370e+05,2.316370e+05,231637.000000,231637.000000,231637.000000
mean,222014.708984,9.398546e+03,5.534885e+06,9.765499,9.051153,4.346246
std,141206.635626,4.461963e+06,9.979141e+07,5.995128,3.734796,0.990806
min,38.000000,0.000000e+00,2.700000e+01,0.000000,1.000000,0.000000
25%,99944.000000,2.000000e+01,5.690500e+04,6.000000,6.000000,4.000000
50%,207249.000000,4.000000e+01,1.736140e+05,9.000000,9.000000,4.714286
75%,333816.000000,6.500000e+01,3.982750e+05,12.000000,11.000000,5.000000
max,537716.000000,2.147484e+09,2.002290e+09,145.000000,43.000000,5.000000


In [82]:
df_merge.head(3)

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,rating
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,5.0
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6,3.5
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13,4.0


In [116]:
no_rating = list(df_merge[df_merge['rating']==0].id)

In [119]:
df_review[df_review['recipe_id'].isin(no_rating)]

,user_id,recipe_id,date,rating,review
119,1232945,487707,2012-10-08,0,Loved this salsa flavored meatloaf...tang is a...
222,249623,25749,2009-06-14,0,Sounds marvelous!!! I wish I could try this. M...
578,2001346860,471641,2017-01-11,0,Great looking recipe/ vegan- no animal product...
608,726381,232036,2008-01-13,0,nothing authentic here ..
789,2000431022,467556,2015-08-26,0,Loved this for breakfast. My husband had to as...
1088,1840822,446948,2011-02-27,0,NOT an oreo lots of time to make and very disa...
1116,112818,159320,2007-02-09,0,I did not like the taste of this recipe. I fee...
1706,181231,36254,2005-09-27,0,You can't beat potato soup on a cold winter da...
2143,38049,29467,2002-09-09,0,"Delicious recipe, however the recipe did not s..."
2586,99325,153263,2006-05-30,0,"This is an interesting recipe, but I don't thi..."


In [30]:
url = "https://www.allrecipes.com/recipes/"

x = url.response.css('::text')

AttributeError: 'str' object has no attribute 'response'

In [25]:
class FoodSpider(scrapy.Spider):
    name = 'food_spider'
    
    #start_requests method
    def start_requests(self):
        urls = "https://www.allrecipes.com/recipes/"
        for url in urls:
            yield scrap.Request( url = url, callback = self.parse )
    
    #parse method
    def parse(self, response):
        html_file = 'food_file.html'
        with open( html_file, 'wb') as fout:
            fout.write(response.body)

In [ ]:
loc = '/Users/brandon.arcilla/Desktop/chromedriver' #location of Chrome webdriver
options = Options()
options.add_argument('--headless')
options.add_argument('--log-level=3')

driver = webdriver.Chrome(loc, chrome_options=options)

In [15]:
driver.get('https://www.allrecipes.com/recipes/')

In [16]:
major_cats = {'World Cuisine'} # names of Allrecipes.com major categories (section headers) to scrape

# initialize df to hold category information
cat_df = pd.DataFrame(columns=['cat', 'subcat', 'link'])

# iterate over sections
for section in driver.find_elements_by_xpath("//div[@class='all-categories-col']//section"):
  section_name = section.find_elements_by_xpath(".//h3[@class='heading__h3']")[0].text
  # skip some categories
  if section_name not in major_cats:
    continue
  # retain the subcategory name and link for the remaining categories 
  cat_df = pd.concat([cat_df, 
    pd.DataFrame([(section_name, a.text,a.get_attribute('href')) 
  for a in section.find_elements_by_xpath(".//ul//li//a")], 
     columns=['cat', 'subcat', 'link'])])

In [17]:
cat_df

,cat,subcat,link
0,World Cuisine,Chinese,https://www.allrecipes.com/recipes/695/world-c...
1,World Cuisine,Indian,https://www.allrecipes.com/recipes/233/world-c...
2,World Cuisine,Italian,https://www.allrecipes.com/recipes/723/world-c...
3,World Cuisine,Mexican,https://www.allrecipes.com/recipes/728/world-c...
4,World Cuisine,Southern,https://www.allrecipes.com/recipes/15876/us-re...
5,World Cuisine,Thai,https://www.allrecipes.com/recipes/702/world-c...
6,World Cuisine,All World Cuisine,https://www.allrecipes.com/recipes/86/world-cu...
